In [1]:
import os
import pandas as pd
import toml

# Specify config 
config_path = "../configs/config_2024.toml"
config = toml.load(config_path)

dict_values(['Passenger Vehicles', 'Medium Trucks', 'Heavy Trucks', 'Bus'])

In [13]:
def process_cities(county):
    """Process VMT for each city in the specified county. Summarize VMT totals and shares by vehicle type.

    Args:
        county (str): Name of the county to process (e.g., "King").

    Returns:
        vmt_df (pd.DataFrame): DataFrame with total VMT by city and vehicle type.
        vmt_share_df (pd.DataFrame): DataFrame with VMT shares by city and vehicle type.
    
    """

    # Load each city's results
    # Loop through each file in interzonal_vmt folder
    # Get list of files in interzonal_vmt folder
    results_df = pd.DataFrame()
    for fname in os.listdir(f"{config['output_root']}/data/city/{county}/interzonal_vmt"):
        city = fname.replace('.csv','')
        df_city = pd.read_csv(f"{config['output_root']}/data/city/{county}/interzonal_vmt/{fname}")
        df_city['Passenger Vehicles'] = df_city[['sov_vmt','hov2_vmt','hov3_vmt','tnc_vmt']].sum(axis=1)
        df_city['Medium Trucks'] = df_city['medium_truck_vmt']
        df_city['Heavy Trucks'] = df_city['heavy_truck_vmt']
        df_city['Bus'] = df_city['bus_vmt']

        total_vmt_df = pd.DataFrame(df_city.sum()[["Passenger Vehicles","Medium Trucks","Heavy Trucks","Bus"]])
        total_vmt_df['city'] = city

        results_df = pd.concat([results_df,total_vmt_df])
    results_df.rename(columns={0:config["base_year"]}, inplace=True)

    # Calculate unincorporated value as county total minus sum of cities
    # Load county totals 
    df = pd.read_excel(r"Y:\Air Quality\King County Emissions Inventory\2024\county_summary_2024.xlsx", sheet_name='VMT', index_col=0)

    df_county = pd.DataFrame(df[config["base_year"]])

    df_incorp_tot = results_df.groupby(results_df.index).sum()[[config["base_year"]]] 
    df = df_incorp_tot.merge(df_county, left_index=True, right_index=True, suffixes=("_incorporated_total","_county_total"))

    # Assume unicorporated area is county total minus sum of cities
    df[f"{config['base_year']}_unincorporated"] = df[f"{config['base_year']}_county_total"] - df[f"{config['base_year']}_incorporated_total"]
    df['city'] = "Unincorporated King County"
    df.rename(columns={f"{config['base_year']}_unincorporated": config['base_year']}, inplace=True)

    results_df = pd.concat([results_df,df[[config["base_year"],"city"]]])

    results_df.rename(columns={f"{config['base_year']}": f"vmt_{config['base_year']}"}, inplace=True)

    results_df.reset_index(inplace=True)
    results_df.rename(columns={"index":"vehicle_type"}, inplace=True)

    # Unstack so vmt_2023 has a column for each vehicle_type
    vmt_col = f"vmt_{config['base_year']}"
    vmt_df = results_df.pivot(index='city', columns='vehicle_type', values=vmt_col).reset_index()
    vmt_df.columns.name = None

    # Calculate shares
    vmt_share_df = vmt_df.copy()
    for col in config["veh_type_map"].values():
        vmt_share_df[col] = vmt_share_df[col]/vmt_share_df[col].sum()

    # Write as Excel spreadsheet
    return vmt_df, vmt_share_df

In [14]:
county = "King"
vmt_df, vmt_share_df = process_cities(county)

In [35]:
col_list = ["city"]+list(config["veh_type_map"].values())
# Write results to spreadsheet with two tabs: totals and shares
with pd.ExcelWriter(f"{config['output_root']}/city_vmt_summary_{county}.xlsx") as writer:
    vmt_share_df[col_list].to_excel(writer, sheet_name='VMT_Shares', index=False)
    vmt_df[col_list].to_excel(writer, sheet_name='VMT_Totals', index=False)

In [36]:
vmt_share_df[col_list].head()

,city,Passenger Vehicles,Medium Trucks,Heavy Trucks,Bus
0,Algona,0.003485,0.005643,0.007818,0.001403
1,Auburn,0.040727,0.042109,0.046842,0.019028
2,Beaux Arts,0.000011,0.000004,0.000000,0.000064
3,Bellevue,0.089051,0.099833,0.059582,0.080669
4,Black Diamond,0.001198,0.000697,0.000318,0.000201


In [37]:
vmt_df[col_list].head()

,city,Passenger Vehicles,Medium Trucks,Heavy Trucks,Bus
0,Algona,1.431767e+05,11044.718470,11088.911600,180.541356
1,Auburn,1.673404e+06,82415.171848,66439.664292,2448.318176
2,Beaux Arts,4.602604e+02,7.189447,0.000000,8.184482
3,Bellevue,3.658955e+06,195393.730478,84509.075523,10379.673160
4,Black Diamond,4.923980e+04,1363.803312,450.960647,25.853677
